In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.dpi'] = 100

Regularization term based on Thierry's idea:
$$f(x, v_{\min}, v_{\max}, \alpha, k) = \mathbb{1}_{\{x < v_{\min}\}} \left( \alpha (v_{\min}-x) \right)^{k}  + \mathbb{1}_{\{x > v_{\max}\}} \left( \alpha (x - v_{\max}) \right)^{k}$$

Regularization term based on sigmoid activation:
$$g(x, v_{\min}, v_{\max}, \alpha) = \frac{1}{1 + e^{\alpha (x - v_{\min})}} + \frac{1}{1 + e^{-\alpha (x - v_{\max})}}$$

In [ ]:
# Define the function
def f(x, v_min: float, v_max: float, alpha: float = 20.0):
    return 1 / (1 + np.exp(alpha*(x - v_min))) + 1 / (1 + np.exp(-alpha*(x-v_max)))

def g(x, v_min: float, v_max: float, alpha: float = 2.0, k: int = 1):
    return np.where(x < v_min, (alpha*(x - v_min)) ** (2*k), 0) + np.where(x > v_max, (alpha*(x - v_max)) ** (2*k), 0)

x = np.linspace(-3, 3, 1000)
y = f(x, -np.pi/2-0.3, np.pi/2+0.3)
z = g(x, -np.pi/2, np.pi/2)

# Plot the function
plt.figure(figsize=(10, 5))
plt.plot(x, y, color='b', label='sigmoid inspired indicator function')
plt.plot(x, z, color='g', label='Thierry inspired indicator function')
plt.vlines(np.array([-1, 1]) * np.pi/2, 0, 1.25, color='r', label='limit angle')
plt.xlim(-3, 3)
plt.xticks(ticks=[-np.pi/2, 0, np.pi/2], labels=[r"$-\frac{\pi}{2}$", r"$0$", r"$\frac{\pi}{2}$"])
plt.ylim(0, 1.25)
plt.legend()
plt.grid()
plt.show()

Instead of:

$$h(x) = \text{mod}_{\pi}(x)^2$$

I propose to use the following loss function that takes into account the periodicity: 
$$h(x, \alpha) = 1 - \exp\left(-\alpha \sin^2\left(x\right)\right)$$

In [ ]:
def loss_mod(x):
    return np.abs((x + np.pi/2) % np.pi - np.pi/2)

def loss_fourier(x):
    nbCoefs = 20
    sum = np.zeros_like(x)
    for i in range(nbCoefs):
        sum += np.cos(2*(2*i+1)*x)/(2*i+1)**2
    y = 1/2 - sum*4/np.pi**2
    return y

def loss_func_periodic(x, alpha):
    return 1 - np.exp(-alpha*np.sin(x)**2)

alpha1 = 5.0
alpha2 = 10.0

x = np.linspace(-2*np.pi-0.5, 2*np.pi+0.5, 1000)

l0 = loss_mod(x)
l1 = loss_func_periodic(x, alpha=alpha1)
l2 = loss_func_periodic(x, alpha=alpha2)
plt.plot(x, l0, color='black', label='mod')
plt.plot(x, l1, color='red', label=r"$\alpha={}$".format(alpha1))
plt.plot(x, l2, color='green', label=r"$\alpha={}$".format(alpha2))
plt.xticks(ticks=[-2*np.pi, -np.pi, 0, np.pi, 2*np.pi], labels=[r"$-2\pi$", r"$-\pi$", r"$0$", r"$\pi$", r"$2\pi$"])
plt.legend()
plt.show()